# RAG Example 

In [ ]:
import os 
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import AzureOpenAI, OpenAI
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import OpenSearchVectorSearch

In [ ]:
loader = DirectoryLoader("./data/", glob="*.txt")
documents = loader.load()

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=10)
texts = text_splitter.split_documents(documents)

In [ ]:
# select which embeddings we want to use
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/distiluse-base-multilingual-cased-v2")

In [ ]:
# create the vectorestore to use as the index
db = OpenSearchVectorSearch.from_documents(
    texts,
    embeddings,
    opensearch_url="https://localhost:9200",
    http_auth=("admin", "admin"),
    use_ssl = False,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)


In [ ]:
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [ ]:
# prepare llm
llm = OpenAI()

In [ ]:
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

In [ ]:
query = "請問企業導入AI案例與建議有什麼"
qa({"query": query})